In [ ]:
import numpy        as np
import pandas       as pd
import dynesty      as dyn
import matplotlib   as mpl
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.special import factorial
from plotutils import addtxt, colorbar
from tqdm import tqdm
from tqdm import trange
from dynesty import plotting as dyplot
import seaborn as sns
from scipy import stats
mpl.style.use(['./scripts/theme_bw.mplstyle', './scripts/presentation.mplstyle'])

\begin{align}
\mathcal{Z} &= \int_0^1 \mathcal{L}(X) \text{d}X \simeq \sum_{k=1}^N A_k \\
A_k &= w_k \mathcal{L}_k \\
w_k &= X_{k-1} - X_k \\
w_k &\simeq e^{-k/n}\qquad (n: \text{number of live points})
\end{align}

\begin{align}
\mathcal{H} &= \int P(X)\log{\left[P(X)\right]} \text{d}X \simeq \sum_{k=1}^N \frac{A_k}{\mathcal{Z}} \log{\left[\frac{\mathcal{L}_k}{\mathcal{Z}}\right]}
\end{align}

In [ ]:
def logAdd(logx,logy):
        """logarithmic addition log(exp(logx) + exp(logy))"""
        if logx > logy:
            return logx + np.log(1.0 + np.exp(logy-logx))
        else:
            return logy + np.log(1.0 + np.exp(logx-logy))

In [ ]:
def fmap(func, *iterables):
    return np.array(list(map(func, *iterables)))

def nestedSampler(*, n, maxIter, newObj, prior, logLikelihood, explorer, mcparams):
    """Adapted from Sivia and Skilling, Data Analysis : A Bayesian Tutorial, pg. 188"""
    
    obj         = newObj(n)                     # live points θ
    logL        = np.zeros(n)                   # log(L) for live points

    sample      = newObj(maxIter)               # dead points θ
    logLsample  = np.zeros(maxIter)             # log(L) for dead points
    logWtsample = np.zeros(maxIter)             # log(weight) for dead points

    H           = 0.0                           # information, initially 0
    logZ        = np.finfo(np.double).min       # log(Evidence Z), initially 0
    logWidth    = np.log(1.0 - np.exp(-1.0/n))  # log(width in prior mass), initially set to outermost interval of prior mass

    obj[...]    = prior(n)                      # sample objects from prior Π(θ)
    logL[...]   = fmap(logLikelihood, obj)      # compute likelihood of all points
    for nest in trange(maxIter):
        worst   = np.argmin(logL)               # find worst object in collection, with Weight = width * likelihood
        logWt   = logWidth + logL[worst]        # weight (width * likelihood) of new dead point
        
        logZnew = logAdd(logZ, logWt)           # update evidence Z and Information H
        H       = np.exp(logWt - logZnew) * logL[worst] + np.exp(logZ - logZnew) * (H + logZ) - logZnew
        logZ    = logZnew

        sample[nest]      = obj[worst]          # add worst point to list of dead points (posterior samples)
        logLsample[nest]  = logL[worst]      
        logWtsample[nest] = logWt
                
        copy = np.random.randint(low=0, high=n) # kill worst object in favor of copy of different survivor
        while copy == worst and n > 1:
            copy = np.random.randint(low=0, high=n)
        
        logLstar    = logL[worst]               # new likelihood constraint
        obj[worst]  = obj[copy]                 # "worst" is now new live point
        explorer(x=obj[worst], logLikelihood=logLikelihood, logLstar=logLstar, params=mcparams)
        logL[worst] = logLikelihood(obj[worst])
        
        logWidth   -= 1.0/n                       # shrink interval

    print(f'Iterate #     = {nest:7d}')
    print(f'Evidence logZ = {logZ:12.6g} +/- {np.sqrt(H/n):8.6g}')
    print(f'Information H = {H:12.6g} nats = {H/np.log(2.0):12.6g} bits')
    return obj, sample, logLsample, logWtsample, H, logZ

# d-dimensional Gaussian

\begin{align}
L(\Theta) &= \exp{\left(-\frac{r^2}{2\sigma^2}\right)}, \qquad r^2 = \Theta^2 = \sum_i^d \theta_i^2
\end{align}

where $\Theta$ has a flat prior within the unit d-ball. Since the volume of a d-ball with radius $R$ is (assuming $d$ is even)
\begin{align}
V_d &= \frac{\pi^{d/2}}{\left(\frac{d}{2}\right)!} R^d ,\qquad (\pi = 3.1415\ldots)
\end{align}

the the prior is
\begin{align}
\Pi(\Theta) &= \frac{\left(d/2\right)!}{\pi^{d/2}} = 1 / V_{d}(R=1)
\end{align}

We take $d = 10$, and $\sigma = 0.01$ so all the likelihood is well within the prior domain. The evidence, discarding the tails outside the domains is

\begin{align}
\mathcal{Z} &= \int \mathcal{L}(\Theta) \Pi(\Theta)\,\text{d}\Theta \\
&\simeq \Pi(\Theta) \int \exp{\left[-\frac{1}{2}\Theta^\text{t} \Sigma^{-1} \Theta\right]} \text{d}\Theta,\qquad \Sigma^{-1} = \frac{1}{\sigma^2} I \\
&= \Pi(\Theta) \sqrt{(2\pi)^d \det\Sigma} \\
&= \frac{(d/2)!}{\pi^{d/2}} (2\pi)^{d/2} \sigma^{d} \\
&= \left(d/2\right)! \left(2 \sigma^2\right)^{d/2} \\
\log{\mathcal{Z}} &= \frac{d}{2}\log{\left(2 \sigma^2\right)} + \log{\left(d/2\right)!}
\end{align}

Since $L$ is a decreasing function of $r$, if we could perform the sorging, we would organize $\theta$ is radial sequences of nested shells. In this case we can exactly compute the prior mass
\begin{align}
X(\lambda) &= \int_{\left\{\mathcal{L}(\Theta) > \lambda\right\}} \Pi(\Theta) \text{d}\Theta \\
&= \Pi(\Theta)\int_{r^2 \le -2\sigma^2\log(\lambda)} \text{d}\Theta \\
&= \Pi(\Theta) V_d\left(R = \sqrt{-2\sigma^2\log{\lambda}}\right) \\
&= (-2\sigma^2\log\lambda)^{d/2}
\end{align}
Thus, we can invert $X(\lambda)$ to yield
\begin{align}
L(X) &= \exp{\left[-\frac{X^{2/d}}{2\sigma^2}\right]}
\end{align}

Recall that what we want to evaluate is
\begin{align}
\mathcal{Z} &= \int_0^1 \mathcal{L}(X) \text{d}X
\end{align}

In [ ]:
def logLikelihood(θ):
    """L(θ)"""
    return -np.sum(θ**2) / (2*σ**2) 

def prior(*, num, dim):
    """Return num points uniformly sampled in unit dim-ball"""
    def marsaglia():
        """Return n points sampled over surface of unit dim-ball"""
        r = np.random.randn(num, dim)
        return r / np.linalg.norm(r, axis=1)[...,None]
    x = marsaglia()
    u = np.random.rand(num)**(1.0/dim)
    x[...] = u[:,None]*x[...]
    return x

def normalize(a):
    return a / np.max(np.abs(a))

def mcexplorerUnitSphere(*, x, logLikelihood, logLstar, params):
    x[:] = np.sqrt(-logLstar*2*σ**2)*prior(num=1, dim=d)[0]

d,σ =10,0.01
def logLikelihoodX(X):
    """L(X), the likelihood countour enclosing prior mass X"""
    return -X**(2/d)/(2*σ**2)

def logPriorMassL(logλ):
    """X(λ)"""
    return d/2*(np.log(2*σ**2) + np.log(-logλ))

logX = np.linspace(-70, 0, num=250)
θ    = prior(num=100, dim=d)             # uniform samples
logL = fmap(logLikelihood, θ)            # likelihood of samples

fig, ax = plt.subplots(figsize=(12,9))
ax.plot(-logX, np.exp(logLikelihoodX(np.exp(logX))), label='prior')
ax.plot(-logX, normalize(np.exp(logLikelihoodX(np.exp(logX)) + logX)), label='posterior')
ax.plot(-logPriorMassL(logL), np.exp(logL), marker='x', ls='None')
ax.legend()
ax.set_xlim(0, 70)
ax.set_xlabel(r'$-\log{X}$')
ax.set_ylabel(r'$\mathcal{L}(X)$')
plt.show()

In [ ]:
npoints  = 1
nsamples = npoints*100
params   = {'δ':0.1, 'steps':100, 'a':-1, 'b':1}
live, sample, logLsample, logWtsample, H, logZ = nestedSampler(n=npoints, maxIter=nsamples, newObj = lambda n: np.zeros((n,d)), prior=lambda n : prior(num=n, dim=d), logLikelihood=logLikelihood, explorer=mcexplorerUnitSphere, mcparams=params)
print('')
print(f'log(Z_theory) = {d/2*np.log(2*σ**2) + np.log(factorial(d//2)):.6f}')
print(f'Z_theory      = {factorial(d//2) * (2*σ**2)**(d//2):.6e}')
fig, ax = plt.subplots(figsize=(12,9))
ax.plot(-logX, np.exp(logLikelihoodX(np.exp(logX))))
ax.plot(np.arange(0,nsamples)/npoints, np.exp(logLsample), marker='x', ls='None')
ax.set_xlim(0,70)
ax.set_xlabel(r'$-\log{X}$')
ax.set_ylabel(r'$\mathcal{L}(X)$')
plt.show()

Let's try same problem, but now using a general MCMC sampler to generate the new points. For this let us slightly change the problem and assume a uniform prior over a hyper-cube (instead of hyper-sphere) with sides (-1,1)
\begin{align}
\mathcal{Z} &= \int \mathcal{L}(\Theta) \Pi(\Theta)\,\text{d}\Theta \\
&= \Pi(\Theta) \int \exp{\left[-\frac{1}{2}\Theta^\text{t} \Sigma^{-1} \Theta\right]} \text{d}\Theta,\qquad \Sigma^{-1} = \frac{1}{\sigma^2} I \\
&= \Pi(\Theta) \sqrt{(2\pi)^d \det\Sigma} \\
&= \frac{1}{2^d}\sqrt{(2\pi)^d \sigma^{2n}} = \left(\frac{\pi}{2}\right)^{d/2} \sigma^n\\
\log{\mathcal{Z}} &= \frac{d}{2}\log{\frac{\pi}{2}} + n\log{\sigma}
\end{align}

In [ ]:
print(f'log(Z_theory) = {d/2*np.log(np.pi/2) + d*np.log(σ):.6f}')
print(f'Z_theory      = {(np.pi/2)**(d//2) * σ**d:.6e}')

In [ ]:
def mcexplorer(*, x, logLikelihood, logLstar, params):
    """MCMC explorer in d-cube over (a,b) """
    def tounitcube(z, lo=params['a'], hi=params['b']): 
        """Map from (lo,hi) to (0,1)"""
        return (z - lo)/(hi-lo)
    def fromunitcube(z, lo=params['a'], hi=params['b']):
        """Map from (0,1) to (lo,hi)"""
        return (hi-lo)*z + lo
    def uniform():
        """Uniform random number [-1,1) """
        return fromunitcube(np.random.rand(len(x)), lo=-1, hi=1)

    u    = tounitcube(x)
    v    = np.zeros_like(x)
    y    = np.zeros_like(x)
    accept, reject = 0, 0
    for i in range(params['steps']):
        v[:] = u[:] + params['δ']*uniform() # new trial point
        v[:]-= np.floor(v)
        y[:] = fromunitcube(v)
        logL = logLikelihood(y)
        if(logL > logLstar):                # accept iff within hard likelihood constraint
            u[:] = v[:]
            x[:] = y[:]
            accept += 1
        else:
            reject += 1
        if accept > reject: # modify step size to get convergence rate of 50%
            params['δ'] *= np.exp(1.0 / accept)
        if accept < reject:
            params['δ'] /= np.exp(1.0 / reject)
        params['δ'] = np.min([params['δ'], (params['b']-params['a'])*0.5])
def prior(*,num,dim):
    return 2.0*np.random.rand(num,dim)-1

In [ ]:
npoints  = 500
nsamples = npoints*50
params   = {'δ':0.2, 'steps':100, 'a':-1, 'b':1}
live, sample, logLsample, logWtsample, H, logZ = nestedSampler(n=npoints, maxIter=nsamples, newObj = lambda n: np.zeros((n,d)), prior=lambda n : prior(num=n, dim=d), logLikelihood=logLikelihood, explorer=mcexplorer, mcparams=params)

Now let's see how to solve the same problem using the "dynesty" package

In [ ]:
def priorTransform(u):
    """Transforms our defualt unit cube samples `u` to a flat prior between -1. and 1. in each variable."""
    return (2. * u - 1.)

# Vanilla nested sampler (as above)
sampler = dyn.NestedSampler(logLikelihood, priorTransform, d, bound='single')
sampler.run_nested(dlogz=0.01)
res     = sampler.results

# Dynamic nested sampler (fancier!)
dsampler = dyn.DynamicNestedSampler(logLikelihood, priorTransform, d, bound='single', sample='unif')
dsampler.run_nested(nlive_init=50, nlive_batch=50, maxiter=res.niter+res.nlive, use_stop=False, wt_kwargs={'pfrac': 0.0})
dres_z = dsampler.results

In [ ]:
fig, axes = dyplot.cornerplot(dres_z, quantiles=None, color='darkviolet',span=[[-.2,.2], [-.2, .2], [-.2,.2], [-.2,.2], [-.2,.2], [-.2,.2], [-.2, .2], [-.2,.2], [-.2,.2], [-.2,.2]],fig=plt.subplots(10, 10, figsize=(30, 30)))
fig.tight_layout()

# Egg-Basket

Define the Egg-box likelihood $\mathcal{L}(x,y) \propto p(D|\theta)$, where $\theta=(x,y)$ are the model parameters, as
\begin{align}
\mathcal{L}(x,y) &= \exp{\left[\left(2 + \cos{\left(x\right)}\cos{\left(y\right)}\right)^5\right]}
\end{align}
with a uniform prior  $U(0,6\pi)$ for both $x$ and $y$. Note the system is periodic in both $x$ and $y$ (period $6\pi$).

The Bayesian evidence integral is
\begin{align}
\mathcal{Z} &= \int \mathcal{L}(\theta)\pi(\theta)\text{d}\theta
= \frac{1}{(6\pi)^2}\int\mathcal{L}(\theta)\text{d}\theta\\
\log\mathcal{Z} &= \log{\left[\int\mathcal{L}(\theta)\text{d}\theta\right]} - 2\log{6\pi}
\simeq 235.856
\end{align}

In [ ]:
def logLikelihood(θ):
    return (2.0 + np.cos(θ[0]) * np.cos(θ[1])) ** 5.0
a,b= 0.0,6.0*np.pi
X,Y= np.meshgrid(np.linspace(a,b,1000),np.linspace(a,b,1000),indexing='ij')
Z  = integrate.dblquad(lambda x,y : np.exp(logLikelihood(np.array([x,y]))), a, b, lambda x: a, lambda x: b)
print(f'Z = {np.log(Z[0]) - 2*np.log(b-a):.3f}')

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
im = ax.pcolormesh(X,Y,logLikelihood(np.stack([X,Y])))
ax.set_xlabel(r'$x$', fontsize=22)
ax.set_ylabel(r'$y$', fontsize=22)
colorbar(ax, im)
plt.show()

In [ ]:
def prior(*,num,dim):
    return 6.0*np.pi*np.random.rand(num,dim)
d        = 2
npoints  = 1000
nsamples = npoints*100
params   = {'δ':0.1, 'steps':100, 'a':a, 'b':b}
live, sample, logLsample, logWtsample, H, logZ = nestedSampler(n=npoints, maxIter=nsamples, newObj = lambda n: np.zeros((n,d)), prior=lambda n : prior(num=n, dim=d), logLikelihood=logLikelihood, explorer=mcexplorer, mcparams=params)

In [ ]:
fig, ax = plt.subplots(figsize=(12,9))
ax.plot(np.arange(0,nsamples)/npoints, np.exp(logLsample), marker='x', ls='None')
ax.set_xlim(0,10)
ax.set_xlabel(r'$-\log{X}$')
ax.set_ylabel(r'$\mathcal{L}(X)$')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
im = ax.pcolormesh(X,Y,logLikelihood(np.stack([X,Y])))
colorbar(ax, im)
ax.plot(sample[:,0], sample[:,1], ls='None', marker='x')
ax.set_xlabel(r'$x$', fontsize=22)
ax.set_ylabel(r'$y$', fontsize=22)
plt.show()

In [ ]:
sns.jointplot(x=sample[:,0], y=sample[:,1], kind="hex", color="k" );

In [ ]:
def priorTransform(u):
    """Transforms our defualt unit cube samples `u` to a flat prior between 0 and 6pi in each variable."""
    return u*6.0*np.pi
dsampler = dyn.DynamicNestedSampler(logLikelihood, priorTransform, ndim=d, bound='multi', sample='unif')

In [ ]:
# focus on deriving the evidence
dsampler.run_nested(dlogz_init=0.01, nlive_init=500, nlive_batch=500,wt_kwargs={'pfrac': 0.0}, stop_kwargs={'pfrac': 0.0})
dres_z = dsampler.results

In [ ]:
# now add samples to get the posterior
dsampler.reset()
dsampler.run_nested(dlogz_init=0.01, nlive_init=500, nlive_batch=500,wt_kwargs={'pfrac': 1.0}, stop_kwargs={'pfrac': 1.0})
dres_p = dsampler.results

In [ ]:
fig, axes = dyplot.runplot(dres_z, color='blue')
fig.tight_layout()

In [ ]:
fig, axes = dyplot.cornerplot(dres_p, quantiles=None, color='darkviolet',span=[[0, 6*np.pi], [0, 6*np.pi]],fig=plt.subplots(2, 2, figsize=(10, 10)))
fig.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
im = ax.pcolormesh(X,Y,logLikelihood(np.stack([X,Y])))
colorbar(ax, im)
ax.plot(dres_z.samples[:,0], dres_z.samples[:,1], ls='None', marker='x')
ax.set_xlabel(r'$x$', fontsize=22)
ax.set_ylabel(r'$y$', fontsize=22)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
im = ax.pcolormesh(X,Y,logLikelihood(np.stack([X,Y])))
colorbar(ax, im)
ax.plot(dres_p.samples[:,0], dres_p.samples[:,1], ls='None', marker='x')
ax.set_xlabel(r'$x$', fontsize=22)
ax.set_ylabel(r'$y$', fontsize=22)
plt.show()